In [1]:

import pgeon.policy_graph as PG
from pathlib import Path
from example.environment import SelfDrivingEnvironment
from example.discretizer.discretizer import AVDiscretizer
from example.discretizer.discretizer_d1 import AVDiscretizerD1
import pandas as pd
from example.transition import TransitionRecorded


In [2]:
env = SelfDrivingEnvironment()
disc = AVDiscretizer()
    # Generate Policy Graph
    #from existing csv file
    #pg = PG.PolicyGraph.from_nodes_and_edges(str(Path(data_folder) / 'nuscenes_nodes.csv'), str(Path(data_folder) / 'nuscenes_edges.csv'), env, env.discretizer  )

    #from raw data
dtype_dict = {
        'modality': 'category',  # for limited set of modalities, 'category' is efficient
        'scene_token': 'str',  
        'steering_angle': 'float64',  
        'timestamp': 'str',  # To enable datetime operations
        'rotation': 'object',  # Quaternion (lists)
        'x': 'float64',
        'y': 'float64',
        'z': 'float64',
        'yaw': 'float64',  
        'velocity': 'float64',
        'acceleration': 'float64',
        #'heading_change_rate': 'float64',
        'delta_local_x': 'float64',
        'delta_local_y': 'float64'
        #'is_destination': 'str'
    }

df = pd.read_csv(Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes') / "train_v1.0-trainval_lidar_0.csv", dtype=dtype_dict, parse_dates=['timestamp'])
pg = PG.PolicyGraph(env, disc)

In [3]:
pg = pg.fit(df, update=False, verbose=False)

Fitting PG from scenes...: 100%|██████████| 668/668 [00:39<00:00, 16.85it/s]

Average Reward: 7.928068862275447 and Standard Deviation: 7.697623982975654 --> Epoch Mean Time: 39.65204048156738


In [4]:
pg._save_csv('/home/saramontese/Desktop/MasterThesis/example/dataset/data/policy_graphs/pg_nodes.csv', '/home/saramontese/Desktop/MasterThesis/example/dataset/data/policy_graphs/pg_edges.csv','/home/saramontese/Desktop/MasterThesis/example/dataset/data/policy_graphs/pg_traj.csv')

# TEST AGENTS

In [5]:
#mode in [PGBasedPolicyMode.GREEDY, PGBasedPolicyMode.STOCHASTIC], \
# node_not_found_mode in [PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM,
#                                      PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES], \
   
random_agent = PG.PGBasedPolicy(pg, mode=PG.PGBasedPolicyMode.GREEDY, node_not_found_mode=PG.PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM)
stochastic_agent = PG.PGBasedPolicy(pg, mode=PG.PGBasedPolicyMode.STOCHASTIC, node_not_found_mode=PG.PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES)


In [7]:
stochastic_agent.test(num_episodes=1, data_file=Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes') / "train_v1.0-trainval_lidar_0.csv",  seed=42, max_steps=40, verbose = True)
diff_aer, diff_std, transferred_learning = stochastic_agent.compare()

---------------------------------
* START TESTING

573.6674119753936 2006.1744084989757 7.771227028161312 0.0907571211037099
Actual state: (573.6674119753936, 2006.1744084989757, 7.771227028161312, 0.0907571211037099)
Action: Action.BRAKE
Actual state: (574.0105118958064, 2009.9444419368383, 7.571227028161312, 0.0907571211037099)
Action: Action.STRAIGHT
Actual state: (574.3536118162192, 2013.714475374701, 7.571227028161312, 0.0907571211037099)
Action: Action.STRAIGHT
Actual state: (574.696711736632, 2017.4845088125635, 7.571227028161312, 0.0907571211037099)
Action: Action.STRAIGHT
Actual state: (575.0398116570448, 2021.254542250426, 7.571227028161312, 0.0907571211037099)
Action: Action.STRAIGHT
No nearest states available.
Actual state: (575.3829115774575, 2025.0245756882887, 7.571227028161312, 0.0907571211037099)
Action: Action.IDLE
No nearest states available.
Actual state: (575.7260114978703, 2028.7946091261513, 7.571227028161312, 0.0907571211037099)
Action: Action.BRAKE_TURN_RIGHT


In [ ]:
random_agent.test(num_episodes=1, data_file = Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes') / "train_v1.0-trainval_lidar_0.csv", seed=42, max_steps=40, verbose = True)
diff_aer, diff_std, transferred_learning = random_agent.compare()

---------------------------------
* START TESTING

Actual state: (427, 1622, 1.794, 0.179)
Action: Action.GAS
Actual state: (427.17751150346106, 1622.9810701637186, 1.994, 0.179)
Action: Action.GAS
Actual state: (427.3728275709604, 1624.0605425504782, 2.194, 0.179)
Action: Action.IDLE
Actual state: (427.56814363845973, 1625.1400149372378, 2.194, 0.179)
Action: Action.IDLE
Actual state: (427.76345970595906, 1626.2194873239973, 2.194, 0.179)
Action: Action.IDLE
Actual state: (427.9587757734584, 1627.298959710757, 2.194, 0.179)
Action: Action.IDLE
Actual state: (428.15409184095773, 1628.3784320975165, 2.194, 0.179)
Action: Action.IDLE
Actual state: (428.34940790845707, 1629.457904484276, 2.194, 0.179)
Action: Action.IDLE
Actual state: (428.5447239759564, 1630.5373768710356, 2.194, 0.179)
Action: Action.IDLE
Actual state: (428.74004004345574, 1631.6168492577951, 2.194, 0.179)
Action: Action.IDLE
Actual state: (428.93535611095507, 1632.6963216445547, 2.194, 0.179)
Action: Action.IDLE
Actual

# TEST POLICY ITERATION